In [2]:
# -*- coding: utf-8 -*-

#声明Notebook类型，必须在引入pyecharts.charts等模块前声明
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

from pyecharts import options as opts
from pyecharts.charts import Funnel, Page
from impala.dbapi import connect

#连接Hadoop数据库
v1 = []
v2 = []
conn = connect(host='192.168.1.7', port=10000, database='sales',auth_mechanism='NOSASL',user='root')
cursor = conn.cursor()
#读取Hadoop数据
sql_num = "SELECT province,ROUND(SUM(profit),2) FROM orders WHERE dt=2019 and region='华东' GROUP BY province"
cursor.execute(sql_num)
sh = cursor.fetchall()
for s in sh:
    v1.append(s[0])
    v2.append(s[1])
    
#画漏斗图
def funnel_label() -> Funnel:
    c = (
        Funnel()
        .add("利润额",
            [list(z) for z in zip(v1, v2)],
            sort_="descending",               #默认是sort_="descending"即从大到小，也可以设置为ascending即反向漏斗
            label_opts=opts.LabelOpts(position="inside"),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="区域利润额比较分析", subtitle="2019年企业经营状况"),
                         toolbox_opts=opts.ToolboxOpts(),
                         legend_opts=opts.LegendOpts(is_show=True)
                        )
    )
    return c

#第一次渲染时候调用load_javasrcript文件
funnel_label().load_javascript()
#展示数据可视化图表
funnel_label().render_notebook()